In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

/opt/anaconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_excel('/Users/jd/trauma_llm/trauma_data/embedding_responses_jan28.xlsx', sheet_name="NON-EXPERT")

total_response = 5

res = dataset['Value 1']
for ii in range(total_response-1):
    res += dataset['Value '+str(ii+2)]
res /= total_response

dataset['score'] = res

In [3]:
dataset.head()

,Response 1,Response 2,Value 1,Value 2,Value 3,Value 4,Value 5,score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,10,1,2,2,3,2.0
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,12,2,3,2,3,2.4
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,13,2,3,3,2,2.6
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...",12,1,3,4,2,2.4
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...",11,2,3,2,1,2.2


In [4]:
# Load dataset
'''dataset = load_dataset("sentence-transformers/stsb", split="train")
#dataset = load_dataset("tabilab/biosses", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")'''

'dataset = load_dataset("sentence-transformers/stsb", split="train")\n#dataset = load_dataset("tabilab/biosses", split="train")\nprint(f"Loaded {len(dataset)} examples. Sample:")\nprint(dataset[0], "\n")'

In [5]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
model_name = 'nomic-embed-text-v2-moe'#'nomic-embed-text-v1.5'
dims = [768]#[64, 256, 512, 768]

for dim in dims:
    print('Doing dimension ', dim)
    file_to_save = 'trauma/nomic_text-'+model_name+'_'+str(dim)+'_non_expert.pickle'
    embeddings = pd.DataFrame()
    sentence1 = []
    sentence2 = []
    embedding1 = []
    embedding2 = []
    cosine_similarity = []
    human_score = []

    model = SentenceTransformer("nomic-ai/"+model_name, trust_remote_code=True, truncate_dim=64)

    print('Extracting embedding and measuring similarity ...')
    time.sleep(1)
    for idx in tqdm(range(len(dataset))):
        first_sentence = dataset["Response 1"][idx]
        emb1 = response = list(model.encode(first_sentence))
        sentence1.append(
            first_sentence
        )
        embedding1.append(
            emb1
        )
        #############################
        second_sentence = dataset["Response 2"][idx]
        emb2 = list(model.encode(second_sentence))
        sentence2.append(
            second_sentence
        )
        embedding2.append(
            emb2
        )
        
        ############################
        cosine_similarity.append(
            1 - cosine(emb1, emb2)
        )
        human_score.append(
            dataset["score"][idx]/5.0#/4.0 #comment out this depending on the data
        )
        
        time.sleep(1)  # respect rate limits!

    embeddings['Sentence 1'] = sentence1
    embeddings['Sentence 2'] = sentence2
    embeddings['Embedding of Sentence 1'] = embedding1
    embeddings['Embedding of Sentence 2'] = embedding2
    embeddings['Cosine similarity'] = cosine_similarity
    embeddings['Human score'] = human_score

    with open(file_to_save, 'wb') as f:
        pickle.dump(embeddings, f)

Doing dimension  768


/Users/jd/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/7710840340a098cfb869c4f65e87cf2b1b70caca/modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


Extracting embedding and measuring similarity ...


100%|██████████| 244/244 [05:37<00:00,  1.38s/it]


In [6]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,"[0.059141543, 0.08435341, 0.010328197, 0.03660...","[0.057072867, 0.015171323, -0.03103643, 0.0183...",0.596082,0.40
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,"[0.06695909, 0.01634268, -0.015910948, -0.0177...","[0.058899127, -0.005753417, -0.022170316, 0.04...",0.454241,0.48
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,"[0.077441424, -0.00037554657, 0.011272862, -0....","[0.06735218, -0.03721028, -0.012768223, 0.0368...",0.343843,0.52
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...","[0.04914354, 0.015866382, -0.003212126, 0.0296...","[0.057035383, 0.0021539722, -0.039034132, 0.04...",0.471331,0.48
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...","[0.06815589, -0.001376202, 0.014968201, -0.009...","[-0.012010266, -0.014603571, 0.010407524, 0.04...",0.550699,0.44
